<a href="https://colab.research.google.com/github/helene-78/Accidents/blob/main/prep_df_utiles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE # Pour avoir 50 % de 0 et 50 % de 1 dans la colonne presence_accident
from sklearn.linear_model import LogisticRegression # Regression logistique
from sklearn import metrics
from sklearn.model_selection import RepeatedKFold #KFold répété, pour 
from sklearn.model_selection import train_test_split

**Fusion des différents dataframes**

In [ ]:
df_accidents = pd.read_csv('caracteristiques-2019.csv', sep=";")
df_usagers = pd.read_csv('usagers-2019.csv', sep=";")
df_lieux = pd.read_csv('lieux-2019.csv', sep=";")
df_vehicules = pd.read_csv('vehicules-2019.csv', sep=";")
df_test = df_accidents.merge(df_usagers.merge(df_lieux.merge(df_vehicules))) #dataframe à traiter

**Pour supprimer les colonnes inutiles à l'analyse de données, supprimer les NaN et avoir une colonne heure au bon format**

In [ ]:
df_utiles = df_test.drop(axis=1, labels=['Num_Acc', 'com', 'adr', 'v1', 'id_vehicule', 'num_veh', 'lartpc', 'larrout', 'occutc', 'v2', 'voie'])
df_utiles = df_utiles.dropna()
df_utiles["hrmn"] = pd.to_datetime(df_utiles["hrmn"], format='%H:%S')
for i in df_utiles.index:
    df_utiles["hrmn"][i] = (int(df_utiles["hrmn"][i].hour))*60 + (int(df_utiles["hrmn"][i].minute))
df_utiles.dtypes

**Création et remplissage d'une colonne "presence_deces" qui vaut 1 en cas d'accident mortel, 0 sinon**

In [ ]:
df_utiles["hrmn"] = df_utiles["hrmn"].astype('int64')
df_utiles["dep"] = df_utiles["dep"].replace(["2B"], "999") #On remplace les codes des départements de la Corse par des int
df_utiles["dep"] = df_utiles["dep"].replace(["2A"], "998")
df_utiles["actp"] = df_utiles["actp"].replace(["A"], "14") #On remplace les actions du piéton non encodées par des chiffres
df_utiles["actp"] = df_utiles["actp"].replace(["B"], "15")
df_utiles["dep"] = df_utiles["dep"].astype('int64')
df_utiles["presence_deces"] = 0
for i in df_utiles.index:
    if df_utiles["grav"][i] == 2:
        df_utiles["presence_deces"][i] = 1
df_utiles.to_csv('df_utiles.csv')